In [2]:
import numpy as np
import pandas as pd
import sklearn.feature_extraction.text as extraction
from langdetect import detect
import time
from google.cloud import translate_v2 as translate
import nltk
import nltk.stem as stem
from nltk import word_tokenize as tokenize
import re
import string
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/Users/hamilton/Desktop/Stuff/GoogleAPIKey/MovieTranslation-f8371579c36e.json'

Preprocessing:
- removing all data whos feature vector is irregular (longer than the column of 44 or something)
- removed all commas in CSV so that it doesn't interfere with the read_csv
- drop features with missing values
- deleted non-movies
- threw out data with multiple parentheses appearences
- translate all titles to english
- removed year and other parentheses from title
- choosing to use snowball(Porter2) stemmer
- remove punctuation

Steps:
- remove the year in title
- remove other punctuation
- translate to english
- stem the title

TO DO:
- Parse our own title

Visualization:
- distribution of languages for movies (maybe keep track of performance for certain language of movie titles)


# Loading  Data

In [1]:
def load_data():
    client = translate.Client()
    stemmer = stem.snowball.EnglishStemmer()    
    table = str.maketrans('','',string.punctuation)
    data = pd.read_csv("./imdbmovies/imdb.csv", nrows=200)
    
    idxRemove = data[np.logical_not(np.isnan(data['Unnamed: 44']))].index
    dataCleaned = data.drop(idxRemove)
    idxNonMovie = dataCleaned[dataCleaned["type"] != 'video.movie'].index
    dataCleaned.drop(idxNonMovie, inplace=True)
    idxTwoParen = dataCleaned[dataCleaned["title"].str.contains("\(.*\(")].index
    dataCleaned.drop(idxTwoParen, inplace=True)

    dataCleaned.drop(columns=["Unnamed: 44", "Unnamed: 45", "Unnamed: 46", "Unnamed: 47"], inplace=True)
    dataCleaned.dropna(axis=0, inplace=True)
    dataCleaned.reset_index(inplace=True)
    dataCleaned.drop(columns=["index", "url", "tid", "fn", "wordsInTitle", "type"], inplace=True)
    dataCleaned.title = dataCleaned.title.apply(lambda title: re.sub(r'\(([ a-zA-Z]*)([0-9]{4})\)', r'\1', title))\
                                    .apply(lambda title: title.translate(table))\
                                    .apply(lambda title: client.translate(title)["translatedText"])\
                                    .apply(lambda title: " ".join([stemmer.stem(word) for word in tokenize(title)]))
    
    X = dataCleaned.iloc[:, :11]
    y = dataCleaned.iloc[:, 11:]
    
    return X, y

In [3]:
start = time.time()
X, y = load_data()
print("Time to load data: {} seconds".format(time.time() - start))

NameError: name 'stemmer' is not defined

In [39]:
print("Labels ", y.columns.to_list())
print("Features ", X.columns.to_list())

Labels  ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'FilmNoir', 'GameShow', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport', 'TalkShow', 'Thriller', 'War', 'Western']
Features  ['title', 'imdbRating', 'ratingCount', 'duration', 'year', 'nrOfWins', 'nrOfNominations', 'nrOfPhotos', 'nrOfNewsArticles', 'nrOfUserReviews', 'nrOfGenre']


In [ ]:
# X.to_csv("./imdbmovies/features.csv", index=False)
# y.to_csv("./imdbmovies/labels.csv", index=False)

In [69]:
X

,title,imdbRating,ratingCount,duration,year,nrOfWins,nrOfNominations,nrOfPhotos,nrOfNewsArticles,nrOfUserReviews,nrOfGenre
0,the vagabond and the child,8.4,40550,3240.0,1921,1,0,19,96,85,3
1,gold rush,8.3,45319,5700.0,1925,2,1,35,110,122,3
2,metropoli,8.4,81007,9180.0,1927,3,4,67,428,376,2
3,the general,8.3,37521,6420.0,1926,1,1,53,123,219,3
4,citi light,8.7,70057,5220.0,1931,2,0,38,187,186,3
...,...,...,...,...,...,...,...,...,...,...,...
192,the downfal,8.3,184234,9360.0,2004,18,17,16,432,532,3
193,oldboy,8.4,245394,7200.0,2003,18,11,19,121,742,3
194,batman begin,8.3,659591,8400.0,2005,9,44,202,6136,2607,3
195,befor sunset,8.1,113580,4800.0,2004,8,12,34,719,467,2


# Text Vectorization

In [49]:
tfidfVector = extraction.TfidfVectorizer()
hashVector = extraction.HashingVectorizer()

In [230]:
text = ["The quick brown fox jumped over the lazy dog.",
        "The dog.",
        "The fox"]
titles = X.title.tolist()

In [71]:
transOrig = tfidfVector.fit_transform(X.title)

In [72]:
transOrig.shape

(197, 385)

In [78]:
transEng = tfidfVectorEng.fit_transform(titlesEng)

In [80]:
print(tfidfVectorEng.transform(['The']))

  (0, 7665)	1.0


In [74]:
tfidfVectorEng.get_stop_words()

In [65]:
tfidfVectorEng.get_feature_names()

['00',
 '000',
 '007',
 '01',
 '10',
 '100',
 '1000',
 '1001',
 '101',
 '102',
 '105',
 '109',
 '10th',
 '11',
 '1138',
 '117',
 '12',
 '120',
 '127',
 '13',
 '13th',
 '14',
 '1408',
 '1453',
 '1492',
 '15',
 '16',
 '17',
 '174',
 '1776',
 '18',
 '180',
 '1812',
 '18th',
 '19',
 '1901',
 '1911',
 '1931',
 '1935',
 '1937',
 '1938',
 '1939',
 '1941',
 '1942',
 '1944',
 '1950',
 '1953',
 '1962',
 '1974',
 '1980',
 '1983',
 '1987',
 '1989',
 '1991',
 '1998',
 '1999',
 '19th',
 '1st',
 '20',
 '200',
 '2000',
 '2001',
 '2002',
 '2004',
 '2006',
 '2009',
 '2011',
 '2012',
 '2019',
 '2022',
 '2046',
 '2077',
 '20th',
 '21',
 '211',
 '21st',
 '22',
 '23',
 '237',
 '23rd',
 '24',
 '25',
 '25th',
 '27',
 '27th',
 '28',
 '2nd',
 '30',
 '300',
 '3000',
 '33',
 '35',
 '36',
 '39',
 '3d',
 '3rd',
 '40',
 '420',
 '42nd',
 '43',
 '45',
 '451',
 '46',
 '47',
 '48',
 '49th',
 '4th',
 '50',
 '51',
 '51st',
 '52',
 '54',
 '5555',
 '56th',
 '57',
 '571',
 '5ecret',
 '5tar',
 '5th',
 '5tory',
 '5ystem',
 '60

In [4]:
lem = stem.WordNetLemmatizer()
stemmer = stem.snowball.EnglishStemmer()

In [5]:
testStr = 'A Beautiful Minds - genius and madness 2019th'
" ".join([stemmer.stem(x) for x in tokenize(testStr) if bool(re.match('^[0-9a-zA-Z]+$', x))])

'a beauti mind genius and mad 2019th'

In [113]:
porter.stem("articles")

'articl'

In [ ]:
tfidfVector.get_feature_names()

In [ ]:
preprocess = tfidfVector.build_preprocessor()

In [ ]:
preprocess('Savage Harvest 2: October Blood')

In [ ]:
display(tfidfVector.vocabulary_)

In [ ]:
display(tfidfVector.idf_)

In [ ]:
display(vectorizer.transform([text[0]]).toarray())